### In-sample table

In [7]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd

### CHANGE THESE ###
methods = ["basic", "moment20", "filter10", "copula10"]
num_scenarios = [100]

final_rows = []

for ns in num_scenarios:
    for method in methods:
        fold =  "/Users/bruker/Documents/Jobb/empire/OpenEMPIRE/Results/run_analysis/ncc3000.0_na0.75_w0.0_wog0.0_pFalse"
        base_folder = f"Results/run_analysis/{method}/{str(ns)}" # Change Results-folder
        print(base_folder)

        rows = []

        for i in range(1, 31):
            objective_value_path = base_folder + "/" + str(i) + "/results_objective.csv"
            df_obj_value = pd.read_csv(objective_value_path)
            obj_value = float(df_obj_value.columns[0].split(":")[1])
            rows.append({"instance": i, "obj_value": obj_value})

        df = pd.DataFrame(rows)
        mean_value = df['obj_value'].mean()
        std_value = df['obj_value'].std()
        rel_std = std_value / mean_value * 100 # %

        final_rows.append({"method": f"{method}_sce{ns}", "obj_value_mean": mean_value, "obj_value_std": std_value, "rel_std (%)": rel_std})

result_df = pd.DataFrame(final_rows)
result_df

Results/run_analysis/basic/100


FileNotFoundError: [Errno 2] No such file or directory: 'Results/run_analysis/basic/100/1/results_objective.csv'

### OOS table

In [ ]:
### CHANGE THESE ###

methods = ["basic", "moment20", "filter10", "copula10"]
num_scenarios = [10, 50, 100]

final_rows = []

for ns in num_scenarios:
    for method in methods:
        base_folder = f"OutOfSample/dataset_europe_agg_v50/{method}_sce{str(ns)}" # Change OOS-folder
        
        rows = []

        for i in range(1, 31):
            oos_value = 0
            num_trees_considered = 0
            for j in range(1, 11):
                try:
                    oos_value_path = base_folder + f"_{i}/oos_tree{j}/out_of_sample_values.csv"
                    df_oos_value = pd.read_csv(oos_value_path)
                    oos_tree_value = float(df_oos_value["Objective value"][0])
                    oos_value += oos_tree_value
                    num_trees_considered += 1
                except FileNotFoundError:
                    continue
            rows.append({"instance": i, "oos_value": oos_value / num_trees_considered})
            
        df = pd.DataFrame(rows)
        mean_value = df['oos_value'].mean()
        std_value = df['oos_value'].std()
        rel_std = std_value / mean_value * 100 # %

        final_rows.append({"method": f"{method}_sce{ns}", "oos_value_mean": mean_value, "oos_value_std": std_value, "rel_std (%)": rel_std})

result_df = pd.DataFrame(final_rows)
result_df

### Generator boxplots

In [ ]:
SGR_TO_COLOR_AND_TEXT = {
    "basic": ["Basic", "#045275"],
    "moment20": ["Moment", "#089099"], 
    "filter10": ["Strata", "#7CCBA2"],
    "copula10": ["Copula-strata", "#F0746E"], 
}

GENERATOR_TO_TEXT = {
    "Windonshore": "Wind onshore",
    "Windoffshore": "Wind offshore", 
    "Solar": "Solar PV",
    "Bio": "Bio",
    "Hydrorun-of-the-river": "Run-of-river hydropower"
}

In [ ]:
# All fossil types
FOSSIL_TYPES = ['Liginite existing',
'Lignite',
'Lignite CCS adv',
'Coal existing',
'Coal',
'Coal CCS adv',
'Gas existing',
'Gas OCGT',
'Gas CCGT',
'Gas CCS adv',
'Oil existing',
#'Bio existing',
#'Bio 10 cofiring',
#'Nuclear',
#'Bio',
#'Geo',
#'Bio 10 cofiring CCS',
'Lignite CCS sup',
'Coal CCS',
'Gas CCS']
#'Waste']

# Remove whitespace from strings in the list
FOSSIL_TYPES = [fossil.replace(" ", "") for fossil in FOSSIL_TYPES]


### CHANGE THESE ###
FOSSIL = False
genType = 'Bio'
methods = ["basic", "moment20", "filter10", "copula10"]
num_scenarios = 100

def get_dataframes(methods, num_scenarios):
    dataframes = []
    for method in methods: 
        # Change Results-folder
        dataframes.append([pd.read_csv(f"NewResults/run_in_sample/dataset_europe_agg_v50/{method}/{str(num_scenarios)}/{i}/results_output_gen.csv") for i in range(1, 31)])
    return dataframes

def aggregate_dataframes(dataframes):
    # Initialize empty DataFrames to store the aggregated installed capacity values for each method and period
    aggregated_data = pd.DataFrame(columns=['Period', 'genInstalledCap_MW'])

    # Iterate through the dataframes for method
    for df in dataframes:
        # Filter the dataframe for GeneratorType genType
        if FOSSIL:
            filtered_data = df[df['GeneratorType'].isin(FOSSIL_TYPES)]
        else: 
            filtered_data = df[df['GeneratorType'] == genType]
        
        # Group the filtered data by period and sum the installed capacity values within each group
        grouped_data = filtered_data.groupby('Period')['genInstalledCap_MW'].sum().reset_index()
        
        # Concatenate the grouped data with the aggregated DataFrame for Method 1
        aggregated_data= pd.concat([aggregated_data, grouped_data], ignore_index=True)
    
    # Convert MW to GW
    aggregated_data['genInstalledCap_GW'] = aggregated_data['genInstalledCap_MW'] / 1000
    return aggregated_data

all_dataframes = get_dataframes(methods, num_scenarios)
aggregated_dataframes = [aggregate_dataframes(dfs) for dfs in all_dataframes]
all_aggregated_data = [aggregated_data['genInstalledCap_GW'].groupby(aggregated_data['Period']).apply(list) for aggregated_data in aggregated_dataframes]

plt.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(10, 8))

# Define positions for each boxplot
positions = [1, 2, 3]

# Define properties for the median lines
medianlineprops = dict(color='black', linewidth=1)

# Define properties for the outlier marks
flierprops = dict(marker='o', markerfacecolor='white', markersize=4)

# Create boxplots for each method and period
for i, data in enumerate(all_aggregated_data):
    plt.boxplot(data, positions=[pos + i*0.15 - 0.225 for pos in positions], widths=0.15, patch_artist=True, boxprops=dict(facecolor=SGR_TO_COLOR_AND_TEXT[methods[i]][1]), medianprops=medianlineprops, flierprops=flierprops)

# Add labels and title
plt.ylabel('Installed capacity (GW)')
plt.title(f'Generator = {'Fossil' if FOSSIL else GENERATOR_TO_TEXT[genType]}')
plt.ylim(0, 450)
plt.grid(axis="y")

# Customize x-axis labels
plt.xticks(range(1, 4), ['2020-2030', '2030-2040', '2040-2050'])

# Create custom legend for different groups (methods)
legend_handles = [mpatches.Patch(color=SGR_TO_COLOR_AND_TEXT[methods[i]][1], label=SGR_TO_COLOR_AND_TEXT[methods[i]][0]) for i in range(len(methods))]
#plt.legend(handles=legend_handles)

# Show/save the plot
plt.savefig(f"Boxplots/{'Fossil' if FOSSIL else genType}-{num_scenarios}-lockedAxis", bbox_inches="tight")
plt.show()

### In-sample boxplots

In [ ]:
### CHANGE THESE ###
methods = ["basic", "moment20", "filter10", "copula10"]
num_scenarios = [100]

def get_objective_values(methods, num_scenarios):
    objective_values = []
    for method in methods:
        obj_by_method = []
        for ns in num_scenarios:
            # Change Results-folder
            dataframes = [pd.read_csv(f"NewResults/run_in_sample/dataset_europe_agg_v50/{method}/{str(ns)}/{i}/results_objective.csv") for i in range(1, 31)]
            instance_values = [float(df_obj_value.columns[0].split(":")[1]) for df_obj_value in dataframes]
            obj_by_method.append(instance_values)
        objective_values.append(obj_by_method)
    return objective_values

all_objective_values = get_objective_values(methods, num_scenarios)

plt.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(10, 8))

# Define positions for each boxplot
positions = [i for i in range(1, len(num_scenarios) + 1)]

# Define properties for the median lines
medianlineprops = dict(color='black', linewidth=1)

# Define properties for the outlier marks
flierprops = dict(marker='o', markerfacecolor='white', markersize=4)

# Create boxplots for each method and period
for i, data in enumerate(all_objective_values):
    plt.boxplot(data, positions=[pos + i*0.15 - 0.225 for pos in positions], widths=0.15, patch_artist=True, boxprops=dict(facecolor=SGR_TO_COLOR_AND_TEXT[methods[i]][1]), medianprops=medianlineprops, flierprops=flierprops)

# Add labels and title
plt.xlabel("Scenarios")
plt.ylabel('Objective function value')
plt.title(f'Test = In-sample')
plt.ylim(1.4*10**12, 1.55*10**12)
plt.grid(axis="y")

# Customize x-axis labels
plt.xticks(range(1, len(num_scenarios) + 1), [ns for ns in num_scenarios])

# Create custom legend for different groups (methods)
legend_handles = [mpatches.Patch(color=SGR_TO_COLOR_AND_TEXT[methods[i]][1], label=SGR_TO_COLOR_AND_TEXT[methods[i]][0]) for i in range(len(methods))]
#plt.legend(handles=legend_handles)

# Show/save the plot
plt.savefig(f"Boxplots/In-sample-{num_scenarios}", bbox_inches="tight")
plt.show()

### OOS boxplots

In [ ]:
### CHANGE THESE ###
methods = ["basic", "moment20", "filter10", "copula10"]
num_scenarios = [100]

def get_objective_values(methods, num_scenarios):
    objective_values = []
    for method in methods:
        obj_by_method = []
        for ns in num_scenarios:
            instance_values = []
            for i in range(1, 31):
                # Change OOS-folder
                base_path = f"OutOfSample/dataset_europe_agg_v50/{method}_sce{str(ns)}_{i}/"
                oos_values_for_inst = []
                for j in range(1, 11):
                    path_to_df = base_path + f"oos_tree{j}/out_of_sample_values.csv"
                    try:
                        df_oos_value = pd.read_csv(path_to_df)
                        oos_value = float(df_oos_value["Objective value"][0])
                        oos_values_for_inst.append(oos_value)
                    except FileNotFoundError:
                        continue
                instance_values.append(sum(oos_values_for_inst) / len(oos_values_for_inst))
            obj_by_method.append(instance_values)
        objective_values.append(obj_by_method)
    return objective_values

all_objective_values = get_objective_values(methods, num_scenarios)

plt.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(10, 8))

# Define positions for each boxplot
positions = [i for i in range(1, len(num_scenarios) + 1)]

# Define properties for the median lines
medianlineprops = dict(color='black', linewidth=1)

# Define properties for the outlier marks
flierprops = dict(marker='o', markerfacecolor='white', markersize=4)

# Create boxplots for each method and period
for i, data in enumerate(all_objective_values):
    plt.boxplot(data, positions=[pos + i*0.15 - 0.225 for pos in positions], widths=0.15, patch_artist=True, boxprops=dict(facecolor=SGR_TO_COLOR_AND_TEXT[methods[i]][1]), medianprops=medianlineprops, flierprops=flierprops)

# Add labels and title
plt.xlabel("Scenarios")
plt.ylabel('Objective function value')
plt.title(f'Test = Out-of-sample')
plt.ylim(1.4*10**12, 1.55*10**12)
plt.grid(axis="y")

# Customize x-axis labels
plt.xticks(range(1, len(num_scenarios) + 1), [ns for ns in num_scenarios])

# Create custom legend for different groups (methods)
legend_handles = [mpatches.Patch(color=SGR_TO_COLOR_AND_TEXT[methods[i]][1], label=SGR_TO_COLOR_AND_TEXT[methods[i]][0]) for i in range(len(methods))]
#plt.legend(handles=legend_handles)

# Show/save the plot
plt.savefig(f"Boxplots/OOS-{num_scenarios}", bbox_inches="tight")
plt.show()

### Legends

In [ ]:
from matplotlib.patches import Patch

legend_elements = [Patch(facecolor=color, label=sgr) for (sgr, color) in SGR_TO_COLOR_AND_TEXT.values()]

fig, ax = plt.subplots(figsize=(20,1))
ax.legend(handles=legend_elements, loc='center', ncols=4, frameon=False, fontsize=20)
plt.axis("off")
plt.savefig("Legends/standard", bbox_inches="tight")
plt.show()